In [1]:
!pip install anthropic

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip inst

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# import Lenny Newsletter - Sheet1.csv
import pandas as pd

df = pd.read_csv("Lenny Newsletter - Sheet1.csv")
df.head()

,Title,Link,Description,Published,Author,Hearts,Comments
0,How Duolingo reignited user growth,https://www.lennysnewsletter.com/p/how-duoling...,The story behind Duolingo's 350% growth accele...,"Feb 28, 2023",JORGE MAZAL,673.0,59.0
1,How the biggest consumer apps got their first ...,https://www.lennysnewsletter.com/p/how-the-big...,Considering every startup confronts this quest...,"May 12, 2020",LENNY RACHITSKY,452.0,29.0
2,You should be playing with GPTs at work,https://www.lennysnewsletter.com/p/you-should-...,20 examples of how people are using custom GPT...,Feb 20,LENNY RACHITSKY,354.0,21.0
3,How to use ChatGPT in your PM work,https://www.lennysnewsletter.com/p/how-to-use-...,Real-life examples (and actual prompts) of how...,NaN,•,362.0,1.0
4,The 10 commandments of salary negotiation,https://www.lennysnewsletter.com/p/negotiating...,"Guest post by Niya Dragova, co-founder of Candor","Sep 24, 2021",LENNY RACHITSKY,157.0,53.0


In [4]:
# urls = list of "Title" and "Link" as a list of tuples
urls = list(zip(df["Title"], df["Link"]))

print(urls[:5])

[('How Duolingo reignited user growth', 'https://www.lennysnewsletter.com/p/how-duolingo-reignited-user-growth'), ('How the biggest consumer apps got their first 1,000 users', 'https://www.lennysnewsletter.com/p/how-the-biggest-consumer-apps-got'), ('You should be playing with GPTs at work', 'https://www.lennysnewsletter.com/p/you-should-be-playing-with-gpts-at'), ('How to use ChatGPT in your PM work', 'https://www.lennysnewsletter.com/p/how-to-use-chatgpt-in-your-pm-work'), ('The 10 commandments of salary negotiation', 'https://www.lennysnewsletter.com/p/negotiating-comp')]


In [5]:
from anthropic import Anthropic
import os

client = Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)

message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello"
                }
            ]
        }
    ]
)
print(message.content[0].text)

Hello! How can I assist you today?


In [6]:
# loop through the files in the folder and summarize each
summaries = {}

# get a list of urls
for url in urls[:5]:
    with open(f"lenny/{url[1].split('/')[-1]}.txt", "r") as file:
        text = file.read()

    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"<text>{text}</text>\nPull out one nonobvious insight from this text. Use quotes where relevant. Be extremely concise"
                    }
                ]
            }
        ]
    )
    summaries[url] = message.content[0].text

# print the summaries
for url, summary in summaries.items():
    print(url)
    print(summary)
    print("\n")

('How Duolingo reignited user growth', 'https://www.lennysnewsletter.com/p/how-duolingo-reignited-user-growth')
One nonobvious insight is that borrowing successful features from other products requires carefully adapting them to your specific context. As the author notes, "We had borrowed successful features from other products, but the wrong way. We had failed to account for how a change in context can impact the success of a feature."


('How the biggest consumer apps got their first 1,000 users', 'https://www.lennysnewsletter.com/p/how-the-biggest-consumer-apps-got')
One nonobvious insight is that most successful consumer apps found their early users from just a single strategy, rather than employing many different tactics. "Most startups found their early users from just a single strategy. A few like Product Hunt and Pinterest found success using a handful. No one found success from more than three."


('You should be playing with GPTs at work', 'https://www.lennysnewsletter.com/p/

In [7]:
#loop through the files in the folder and add all the text together in one file, separated by a divider ------------
all_text = ""
for url in urls[:50]:
    with open(f"lenny/{url[1].split('/')[-1]}.txt", "r") as file:
        text = file.read()
        all_text += "<article>" + text + "</article>\n------------------------------------\n"

# how many tokens based on length of the text. 1 token is 4 characters
print(len(all_text) / 4)

with open("lenny/all_text.txt", "w") as file:
    file.write(all_text)

185426.25


In [8]:
# summarize the combined text
with open("lenny/all_text.txt", "r") as file:
    text = file.read()

message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"{text}\nSummarize these articles, analyze the word choice, identify any patterns, estimate the reading level, and tell me any other non obvious insights about this publication."
                }
            ]
        }
    ]
)

print(message.content[0].text)

I analyzed the 40 articles from Lenny's Newsletter that you provided. Here is a summary of my findings:

Content Summary:
The articles cover a wide range of topics related to product management, growth, career development, and entrepreneurship. Common themes include:

- Tactical advice and frameworks for product development, prioritization, strategy, pricing, and growth 
- Inside stories of how top companies like Netflix, Figma, Notion, Shopify, and Duolingo build and grow products
- Interviews with experienced leaders sharing career lessons, productivity tips, and insights on leadership
- Guidance for early-stage startups on ideation, validation, finding product-market fit, and scaling
- Personal stories of career journeys, lessons learned from failure, and principles for success

The content is highly practical, example-driven, and grounded in the real-world experience of the author Lenny Rachitsky and the many experts he interviews and features. The articles go deep on specific topi

In [9]:
titles = "\n".join(df["Title"].tolist())

prompt_template = f"""Successful titles follow templates. Here are examples of templates:

Template: "The Blank of Blank"
Description: The format is where the first blank is a word not usually associated with the topic, and the second blank is the topic of the content.
Example 1 - The War of Art
Example 2 - The Psychology of Money
Example 3 - The Subtle Art of Not Giving a Fuck

Template: "Odd Topic"
Description: The first word is an odd adjective to pair with the topic of the content, and the second word is the topic of the content.
Example 1 - Atomic Habits
Example 2 - Extreme Ownership
Example 3 - Deep Work

Template: "The Definitive Guide"
Description: The first word is an unusual adjective not normally associated with the topic of the content, and the second word is the topic of the content.
Example 1 - WebAssembly: The Definitive Guide
Example 2 - Cassandra: The Definitive Guide
Example 3 - Kafka: The Definitive Guide, 2nd Edition

Categorize the following titles into templates. Make up new templates based on what you find. Titles can be assigned to multiple templates. Only list a maximum of 3 examples of titles that fit each template, but skip the template if there are not at least 2 examples. Ignore edition numbers or versions in your analysis. Do not fit a title to a template if it doesn't apply. The example templates may not be suitable for these titles.

Titles:
{ titles }
"""

message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_template
                }
            ]
        }
    ]
)

print(message.content[0].text)

Here are the templates I found based on analyzing the given titles:

Template: "How X Does Y"
Description: The title explains how a specific company or product (X) accomplishes a certain goal or process (Y).
Examples:
- How Duolingo reignited user growth
- How Linear builds product
- How Shopify builds product

Template: "How to X"
Description: The title provides instructions or advice on how to achieve a specific outcome (X).
Examples:
- How to use ChatGPT in your PM work
- How to validate your B2B startup idea
- How to pass any first-round interview (even in a terrible talent market)

Template: "The X of Y" / "The X Y"
Description: The first word (X) is an adjective or noun describing an important aspect, quality, or guide related to the main topic (Y).
Examples:
- The 10 commandments of salary negotiation
- The ultimate guide to willingness-to-pay
- The unconventional Palantir principles that catalyzed a generation of startups

Template: "X for Y"
Description: The title indicates th